<a href="https://colab.research.google.com/github/khaledaldhaheri91-maker/khaled1/blob/main/neural_network_with_lr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a great request. Drawing on the uses cases mentioned in the video sources, I will focus on the example of **e-commerce product recommendations** and generate synthetic data and corresponding TensorFlow/Keras code.

The sources highlight that TensorFlow is an open-source library created by Google used to **build, train, and deploy** machine learning models at a massive scale. For beginners or general developers, the recommended path is using **Keras**, the user-friendly, high-level API built right on top of TensorFlow. The Keras workflow is simply three steps: define, compile, and train.

## 1. Synthetic Large Data Creation: E-commerce Product Recommendations

E-commerce sites use TensorFlow for product recommendations. To model this, we will create a dataset representing user interactions (ratings) with products. Since the query asks for "synthetic large data," we will generate 100,000 interaction records.

*Note: While the source material focuses on TensorFlow and Keras, standard Python libraries like `numpy` and `pandas` (which are not explicitly mentioned in the sources) are required to generate and structure this large synthetic dataset.*

### Use Case: Simplified Collaborative Filtering Model

We simulate data based on:
1.  **Number of Users:** 1,000
2.  **Number of Products:** 500
3.  **Total Interactions (Ratings):** 100,000


## Notebook Roadmap

To keep things clear, this notebook is structured into four main steps:

1. **Generate synthetic recommendation data** (users, products, ratings).
2. **Prepare train/test splits** that a real ML workflow would use.
3. **Build a simple neural recommendation model in Keras** using embeddings.
4. **Train the model with an explicit learning rate** and then **evaluate** it on unseen data.

> Tip: Run the cells **from top to bottom** without skipping to avoid missing variables.


In [ ]:

import numpy as np
import pandas as pd

# Define parameters for 'large' synthetic data
NUM_USERS = 1000
NUM_PRODUCTS = 500
NUM_INTERACTIONS = 100000

# 1. Create Synthetic IDs
# Generate a large array of random User IDs
user_ids = np.random.randint(1, NUM_USERS + 1, NUM_INTERACTIONS)
# Generate a large array of random Product IDs
product_ids = np.random.randint(1, NUM_PRODUCTS + 1, NUM_INTERACTIONS)
# Generate interaction ratings (e.g., 1 to 5 stars)
ratings = np.random.randint(1, 6, NUM_INTERACTIONS)

# 2. Structure the data into a DataFrame
data = pd.DataFrame({
    'user_id': user_ids,
    'product_id': product_ids,
    'rating': ratings
})

# Display the size and a sample of the synthetic data
print(f"--- Synthetic Data for E-commerce Recommendations ---")
print(f"Total Interactions Generated: {len(data)} (Large Scale Data)")
print(data.head())

# 3. Prepare data for model training
# Determine unique counts for embedding layers
unique_users = data['user_id'].nunique()
unique_products = data['product_id'].nunique()

# Map IDs to contiguous integers starting from 0 (required for embeddings)
user_map = {id: i for i, id in enumerate(data['user_id'].unique())}
product_map = {id: i for i, id in enumerate(data['product_id'].unique())}

data['user'] = data['user_id'].map(user_map)
data['product'] = data['product_id'].map(product_map)

# Define X (inputs) and y (target)
X = data[['user', 'product']].values
y = data['rating'].values

# Split data into training and testing sets (standard preparation step)
from sklearn.model_selection import train_test_split # Note: sklearn is not mentioned in sources
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Extract inputs for the model
train_user_input = X_train[:, 0]
train_product_input = X_train[:, 1]


## 2. TensorFlow/Keras Code for the Recommendation Model

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense
from tensorflow.keras.models import Model


--- Synthetic Data for E-commerce Recommendations ---
Total Interactions Generated: 100000 (Large Scale Data)
   user_id  product_id  rating
0      390         281       5
1      424         202       2
2      831         469       3
3      567         199       5
4      903          19       5


In [ ]:

# --- Step 2: Build & Train the Neural Recommendation Model ---

# We assume the previous cell has already created:
# - NUM_USERS, NUM_PRODUCTS          (overall dataset dimensions)
# - train_user_input, train_product_input, y_train
# - X_test, y_test                   (for final evaluation in the later cell)
#
# Here we focus on:
# 1. Defining the model architecture.
# 2. Choosing an optimizer and **learning rate**.
# 3. Training the model while monitoring validation performance.

from tensorflow import keras
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense
from tensorflow.keras.models import Model

# ----------------------------
# 1) Model Hyperparameters
# ----------------------------

EMBEDDING_SIZE = 50   # Size of the latent representation for users/products
LEARNING_RATE = 0.001 # <<-- Key hyperparameter we are adding explicitly

print(f"Using embedding size = {EMBEDDING_SIZE}")
print(f"Using learning rate  = {LEARNING_RATE}")

# ----------------------------
# 2) Input & Embedding Layers
# ----------------------------

# Input layers (each interaction has exactly one user_id and one product_id)
user_input = Input(shape=(1,), name="user_input")
product_input = Input(shape=(1,), name="product_input")

# Embedding layers learn a dense vector for each user and each product.
# We add +1 to NUM_USERS / NUM_PRODUCTS so that the maximum ID is included safely.
user_embedding = Embedding(
    input_dim=NUM_USERS + 1,
    output_dim=EMBEDDING_SIZE,
    name="user_embedding",
)(user_input)

product_embedding = Embedding(
    input_dim=NUM_PRODUCTS + 1,
    output_dim=EMBEDDING_SIZE,
    name="product_embedding",
)(product_input)

# Flatten the embeddings so we can concatenate them
user_vec = Flatten(name="user_vector")(user_embedding)
product_vec = Flatten(name="product_vector")(product_embedding)

# Concatenate user and product representations
concat = Concatenate(name="user_product_concat")([user_vec, product_vec])

# ----------------------------
# 3) Dense Layers (Prediction Head)
# ----------------------------

hidden = Dense(64, activation="relu", name="dense_1")(concat)
hidden = Dense(32, activation="relu", name="dense_2")(hidden)

# Final output layer: predict a single rating (regression)
output = Dense(1, activation="linear", name="rating_output")(hidden)

model = Model(inputs=[user_input, product_input], outputs=output, name="simple_recommender_model")

model.summary()

# ----------------------------
# 4) Compile the Model
# ----------------------------

# Adam optimizer with an explicit learning rate
optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)

model.compile(
    optimizer=optimizer,
    loss="mse",           # Mean Squared Error is a common loss for rating prediction
    metrics=["mae"]       # Track Mean Absolute Error as a human-friendly metric
)

# ----------------------------
# 5) Train the Model
# ----------------------------

history = model.fit(
    x=[train_user_input, train_product_input],  # Inputs
    y=y_train,                                  # Target ratings
    epochs=5,                                   # Number of passes over the training data
    batch_size=256,
    validation_split=0.1,                       # 10% of training data used for validation
    verbose=1
)

print("Model training complete.")

# You can inspect the training curves later via `history.history`


Using embedding size = 50
Using learning rate  = 0.001


Model: "simple_recommender_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ product_input       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ user_embedding      │ (None, 1, 50)     │     50,050 │ user_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ product_embedding   │ (None, 1, 50)     │     25,050 │ product_input[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ user_vector         │ (None, 50)        │          0 │ user_embedding[0… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ product_vector      │ (None, 50)        │          0 │ product_embeddin… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ user_product_concat │ (None, 100)       │          0 │ user_vector[0][0… │
│ (Concatenate)       │                   │            │ product_vector[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 64)        │      6,464 │ user_product_con… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 32)        │      2,080 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rating_output       │ (None, 1)         │         33 │ dense_2[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 83,677 (326.86 KB)

 Trainable params: 83,677 (326.86 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 4.8391 - mae: 1.7966 - val_loss: 2.0897 - val_mae: 1.2508
Epoch 2/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 2.0118 - mae: 1.2242 - val_loss: 2.0582 - val_mae: 1.2384
Epoch 3/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 1.9784 - mae: 1.2104 - val_loss: 2.0507 - val_mae: 1.2347
Epoch 4/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1.9700 - mae: 1.2079 - val_loss: 2.0493 - val_mae: 1.2336
Epoch 5/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.9651 - mae: 1.2081 - val_loss: 2.0554 - val_mae: 1.2365
Model training complete.


In [ ]:
# --- Step 4: Final Evaluation on Unseen Test Data ---

print("\n--- Model Evaluation on Unseen Test Set ---")

# Extract inputs for the model from X_test
test_user_input = X_test[:, 0]
test_product_input = X_test[:, 1]

# The evaluation process assesses the performance of the trained model
# on the independent dataset (X_test, y_test) that the model has never seen.
results = model.evaluate(
    x=[test_user_input, test_product_input], # Test input data
    y=y_test,                                # Test target data
    batch_size=256,
    verbose=0
)

# Extract and display the metrics defined during the compile step
loss = results[0]
mae = results[1] # Mean Absolute Error (the chosen evaluation metric)

print(f"Test Loss (Mean Squared Error): {loss:.4f}")
print(f"Test Mean Absolute Error (MAE): {mae:.4f}")


--- Model Evaluation on Unseen Test Set ---
Test Loss (Mean Squared Error): 2.0224
Test Mean Absolute Error (MAE): 1.2239


### Enhancing Understanding

TensorFlow provides an entire ecosystem. If we were to continue this workflow:

*   **Visualization:** While the model is training, we would typically use **TensorBoard** to visualize the accuracy and spot problems, acting as a dashboard to see the model's mind at work.
*   **Deployment:** Once training is complete, the model would be optimized using **TensorFlow Lite (TFLite)**. TFLite specializes in making the powerful model super small and efficient so it can run on edge devices like smartphones or smart home gadgets, allowing the AI to run right on the device for speed and privacy.

The Keras workflow—defining layers, compiling, and calling `fit`—is like using **Lego bricks** to build a structure. The Lego bricks are the layers (Input, Embedding, Dense), and the compiled model is the instruction manual telling the structure how to stand up and function (the optimizer and loss function). When you call `fit`, you are simply starting the building process with the training data.